In [7]:
!git clone https://github.com/FacerAin/Kor_Eng_NMT_System.git

Cloning into 'Kor_Eng_NMT_System'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 14 (delta 0), reused 10 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), done.


## NMT_seq2seq_basic

Character Level NMT based on Simple Seq2Seq Model Without Attention Mechanism 

Refer to https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py

In [8]:
import numpy as np 

In [9]:
# Parameter Setting
dataset_dir_path = '../dataset/kor-eng/'


In [10]:
# Text data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [11]:
# Read Dataset and Preprocessing
with open(dataset_dir_path+"kor.txt", "r", encoding='utf-8') as f:
    lines = f.read().split('\n')
    for line in lines[:len(lines)-1]:
        split_line = line.split('\t')
        input_texts.append(split_line[0])
        target_texts.append('\t' + split_line[1]+ '\n')
        for char_item in split_line[0]:
            input_characters.add(char_item)
        for char_item in split_line[1]:
            target_characters.add(char_item)

FileNotFoundError: ignored

In [5]:
input_characters = sorted(input_characters)
target_characters = sorted(target_characters)

encoder_token_num = len(input_characters)
target_token_num = len(target_characters)

max_encoder_seq_length = max([len(item) for item in input_texts])
max_decoder_seq_length = max([len(item) for item in target_texts])

encoder_token_index_dict = dict([(char_item, i) for i, char_item in enumerate(input_characters)])
target_token_index_dict = dict([(char_item, i) for i, char_item in enumerate(target_characters)])

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, encoder_token_num), dtype='float32')
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, target_token_num), dtype='float32')
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, target_token_num), dtype='float32')

ValueError: ignored

In [6]:
for i,(input_text, target_text) in enumerate(zip(input_texts,target_texts)):
    for j, char_item in enumerate(input_text):
        encoder_input_data[i, j, encoder_token_index_dict[char_item]] = 1.
    encoder_input_data[i, j+1:, encoder_token_index_dict[" "]] = 1.
    for j, char_item in enumerate(target_text):
        decoder_input_data[i, j, target_token_index_dict[char_item]] = 1.
        if j > 0:
            decoder_target_data[i , j-1, target_token_index_dict[char_item]] = 1.
        decoder_input_data[i, j+1: ,target_token_index[" "]] = 1.
        decoder_target_data[i, j:, target_token_index[" "]] = 1.

        
            

